In [89]:
import msprime
import numpy as np
import matplotlib.pyplot as plt
import math
import tskit
import scipy

In [41]:
r_chrom = 1e-8 #Recombination rate
r_break = math.log(2) #Recombination rate needed to satisfy probability 2^-t inheritance of two chromsomes
chrom_positions = [0, 1e6, 2e6, 3e6] #1Mb chromosome sizes
map_positions = [
    chrom_positions[0],
    chrom_positions[1],
    chrom_positions[1] + 1,
    chrom_positions[2],
    chrom_positions[2] + 1,
    chrom_positions[3]
]
rates = [r_chrom, r_break, r_chrom, r_break, r_chrom] 
rate_map = msprime.RateMap(position=map_positions, rate=rates) #Rate map for separate chromosomes

In [42]:
alpha = np.random.uniform(low=1.05, high=2) #Draw alpha parameter from uniform distribution
Ne = np.random.uniform(low=1000, high=1000000)
ts = msprime.sim_ancestry(
    samples=38,
    population_size=10000,
    recombination_rate=rate_map,
    model=msprime.BetaCoalescent(alpha=alpha),
    random_seed=1234,
)
ts

In [43]:
mts = msprime.sim_mutations(ts, rate=1e-8, random_seed=5678)


In [44]:
np.set_printoptions(legacy="1.21")
summary_statistics = [] #Initialize list of summary statistics
summary_statistics.append(1) #First column corresponds to model index
summary_statistics.append(10000) #Second column is Ne
summary_statistics.append(alpha) #Third column is alpha parameter
summary_statistics.append(1) #Fourth column is rho/theta
S = mts.get_num_mutations()
summary_statistics.append(S) #Fifth column is number of segregating sites
normalized_S = mts.segregating_sites(span_normalise=True)
summary_statistics.append(normalized_S) #Sixth column is span normalized S
pi = mts.diversity()
summary_statistics.append(pi) #Seventh column is nucleotide diversity
summary_statistics

[1,
 10000,
 1.650492657229618,
 1,
 1196,
 0.0003986666666666667,
 7.610374269005781e-05]

In [45]:
afs = mts.allele_frequency_spectrum(span_normalise=False, polarised=False)

afs_entries = []

for x in range(1, 40):
   num_mutations = afs[x]
   l = [x/76] * int(num_mutations)
   afs_entries.extend(l)
afs_entries = np.array(afs_entries)

In [46]:
afs_quant = np.quantile(afs_entries, [0.1, 0.3, 0.5, 0.7, 0.9])
summary_statistics.append(afs_quant[0]) #8th column is AFS quantile 0.1
summary_statistics.append(afs_quant[1]) #9th column 0.3
summary_statistics.append(afs_quant[2]) #10th column 0.5
summary_statistics.append(afs_quant[3]) #11th column 0.7
summary_statistics.append(afs_quant[4]) #12th column 0.9
summary_statistics

[1,
 10000,
 1.650492657229618,
 1,
 1196,
 0.0003986666666666667,
 7.610374269005781e-05,
 0.013157894736842105,
 0.02631578947368421,
 0.06578947368421052,
 0.17105263157894737,
 0.39473684210526316]

In [47]:
num_windows = 30
D_array = mts.Tajimas_D(windows=np.linspace(0, ts.sequence_length, num_windows + 1))
summary_statistics.append(np.nanmean(D_array))
summary_statistics.append(np.nanvar(D_array))
summary_statistics

[1,
 10000,
 1.650492657229618,
 1,
 1196,
 0.0003986666666666667,
 7.610374269005781e-05,
 0.013157894736842105,
 0.02631578947368421,
 0.06578947368421052,
 0.17105263157894737,
 0.39473684210526316,
 -0.21649663539611996,
 0.40273061993611586]

In [48]:
ts_chroms = []
for j in range(len(chrom_positions) - 1):
    start, end = chrom_positions[j: j + 2]
    chrom_ts = mts.keep_intervals([[start, end]], simplify=False).trim()
    ts_chroms.append(chrom_ts)
    print(chrom_ts.sequence_length)

1000000.0
1000000.0
1000000.0


In [49]:

ld_calc = tskit.LdCalculator(ts_chroms[0])
r2_chrom1 = ld_calc.r2_matrix()
r2_chrom1 = np.matrix.flatten(r2_chrom1)
ld_calc = tskit.LdCalculator(ts_chroms[1])
r2_chrom2 = ld_calc.r2_matrix()
r2_chrom2 = np.matrix.flatten(r2_chrom2)
ld_calc = tskit.LdCalculator(ts_chroms[2])
r2_chrom3 = ld_calc.r2_matrix()
r2_chrom3 = np.matrix.flatten(r2_chrom3)
r2 = np.concatenate((r2_chrom1,r2_chrom2,r2_chrom3))
r2_quant = np.quantile(r2, [0.1,0.3,0.5,0.7,0.9])
r2_quant

array([0.00036036, 0.0015015 , 0.0054321 , 0.01647059, 0.0786138 ])

In [50]:
summary_statistics.append(r2_quant[0])
summary_statistics.append(r2_quant[1])
summary_statistics.append(r2_quant[2])
summary_statistics.append(r2_quant[3])
summary_statistics.append(r2_quant[4])
summary_statistics.append(np.mean(r2))
summary_statistics.append(np.var(r2))
summary_statistics

[1,
 10000,
 1.650492657229618,
 1,
 1196,
 0.0003986666666666667,
 7.610374269005781e-05,
 0.013157894736842105,
 0.02631578947368421,
 0.06578947368421052,
 0.17105263157894737,
 0.39473684210526316,
 -0.21649663539611996,
 0.40273061993611586,
 0.0003603603603603603,
 0.001501501501501501,
 0.005432098765432099,
 0.016470588235294122,
 0.07861380242332622,
 0.038414260220286864,
 0.013895439108151025]

In [51]:
for x in range(3):
    with open("output"+str(x+1)+".vcf", "w") as vcf_file:
        ts_chroms[x].write_vcf(vcf_file, contig_id=str(x+1))

In [52]:
import subprocess
import os
os.getcwd()

'/Users/milesanderson/PhD/ILDsim/ILDsim'

In [ ]:
commands = ["bcftools concatoutput1.vcf output2.vcf output3.vcf -o concat.vcf"]

subprocess.run(["bcftools", "concat", "output1.vcf", "output2.vcf", "output3.vcf", "-o", "concat.vcf"])

In [55]:
import pandas as pd

In [75]:
circos = pd.read_csv("glwd.circos", sep= "\t", header=None)
ild_array = circos[circos.columns[6]].to_numpy()

In [85]:
chrom1_S = ts_chroms[0].get_num_mutations()
chrom2_S = ts_chroms[1].get_num_mutations()
chrom3_S = ts_chroms[2].get_num_mutations()

pairwise_num_ild_loci = (chrom1_S*chrom2_S)+(chrom1_S*chrom3_S)+(chrom2_S*chrom3_S)
empty_array_size = pairwise_num_ild_loci - ild_array.shape[0]
empty_array = np.zeros(empty_array_size)
ild_all = np.concatenate((ild_array, empty_array))

In [95]:
np.quantile(ild_all, [0.1,0.3,0.5,0.7,0.9,.95,.99,.9999])

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [93]:
np.mean(ild_array)
#np.var(ild_all)
#scipy.stats.hmean(ild_array)

0.11465800000000001